d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Data Science Project - Ken Wood
### 7/2/21

**Objective**: *Design, complete, and assess a common data science project.*

In this lab, you will use the data science process to design, build, and assess a common data science project.

In [0]:
%run "../../Includes/Classroom-Setup"

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Out[32]: DataFrame[]

res7: Boolean = false

res8: Boolean = false

-sandbox
## Project Details

In recent months, our health tracker company has noticed that many customers drop out of the sign-up process when they have to self-identify their exercise lifestyle (`ht_users.lifestyle`) – this is especially true for those with a "Sedentary" lifestyle. As a result, the company is considering removing this step from the sign-up process. However, the company knows this data is valuable for targeting introductory exercises and they don't want to lose it for customers that sign up after the step is removed.

In this data science project, our business stakeholders are interested in identifying which customers have a sedentary lifestyle – specifically, they want to know if we can correctly identify whether somebody has a "Sedentary" lifestyle at least 95 percent of the time. If we can meet this objective, the organization will be able to remove the lifestyle-specification step of the sign-up process *without losing the valuable information provided by the data*.

There are no solutions provided for this project. You will need to complete it independently using the guidance detailed below and the previous labs from the project.

-sandbox

## Exercise 1

Summary: 
* Specify the data science process question. 
* Indicate whether this is framed as a supervised learning or unsupervised learning problem. 
* If it is supervised learning, indicate whether the problem is a regression problem or a classification problem.

**Hint:** When we are interested in predicting something, we are usually talking about a supervised learning problem.

* **Answer:** The data science question is: given a particular user's data, can we <span style="text-decoration: underline">correctly</span> classify them as having a "Sendentary" or "Non-Sendentary" lifestyle at least 95% of the time?  Since we will be working with labeled data (i.e. the "lifestyle" feature) and we are trying to assign a classification to each user, this is a **supervised learning/classification** problem.

-sandbox

## Exercise 2

Summary: 

* Specify the data science objective. 
* Indicate which evaluation metric should be used to assess the objective.

**Hint:** Remember, the data science objective needs to be measurable.

* **Answer:** The data science objective is to train and test a machine learning classification model that will input user data and classify his/her lifestyle as either "Sendentary" or "Non-Sendentary". We need our model's accuracy to be 0.95 minimum on both the training and test datasets.

-sandbox

## Exercise 3

Summary:
* Design a baseline solution.
* Develop a baseline solution – be sure to split data between training for development and test for assessment.
* Assess your baseline solution. Does it meet the project objective? If not, use it as a threshold for further development.

**Hint:** Recall that baseline solutions are meant to be easy to develop.

In [0]:
%python
import numpy as np

# Create the 'ht_users' df in Pandas...
ht_users_spark_df = spark.read.table("ht_users")
ht_users_pandas_df = ht_users_spark_df.toPandas()

# Create the 'ht_agg' df in Pandas...
ht_agg_spark_df = spark.read.table("ht_agg3")
ht_agg_pandas_df = ht_agg_spark_df.toPandas()

# Create column 'lifestyle_recode' that recodes the 'lifestyle' column
# to either "Sedentary" or "Non-Sedentary"

# ht_users_pandas_df['lifestyle_recode'] = np.where(ht_users_pandas_df['lifestyle'] == "Sedentary", "Sedentary", "Non-Sedentary")
ht_users_pandas_df['lifestyle_recode'] = ht_users_pandas_df['lifestyle'].apply(lambda x: 0 if x == 'Sedentary' else 1)

# Drop 'first_name' and 'last_name' and 'lifestyle' columns they don't
# contribute anything to the analysis...
ht_users_pandas_df.drop(['first_name','last_name','lifestyle'],axis=1,inplace=True)

ht_users_pandas_df['lifestyle_recode'].value_counts()


Out[48]: 1 2688
0 312
Name: lifestyle_recode, dtype: int64

In [0]:
# Create the X matrix and y vector...
X = ht_users_pandas_df[['device_id','country']]
y = ht_users_pandas_df['lifestyle_recode']

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

# Check the baseline percentages for 'y_train'

lifestyle_summary = pd.DataFrame(y_train.value_counts()).reset_index().rename(columns={"index": "lifestyle", "lifestyle_recode": "freq"})
lifestyle_summary['pct'] = (lifestyle_summary['freq'] / lifestyle_summary['freq'].sum()) * 100
lifestyle_summary

Out[50]:

,lifestyle,freq,pct
0,1,2157,89.875
1,0,243,10.125


In [0]:
# Check the baseline percentages for 'y_test'

lifestyle_summary = pd.DataFrame(y_test.value_counts()).reset_index().rename(columns={"index": "lifestyle", "lifestyle_recode": "freq"})
lifestyle_summary['pct'] = (lifestyle_summary['freq'] / lifestyle_summary['freq'].sum()) * 100
lifestyle_summary

Out[51]:

,lifestyle,freq,pct
0,1,531,88.5
1,0,69,11.5


-sandbox
## Conclusion - Exercise 3

Our baseline model shows that, for both the train and test datasets, users with a "Sedentary" lifestyle comprise approximately 10% - 12% of the population.

-sandbox

## Exercise 4

Summary: 
* Design the machine learning solution, but do not yet develop it. 
* Indicate whether a machine learning model will be used. If so, indicate which machine learning model will be used and what the label/output variable will be.

**Hint:** Consider solutions that align with the framing you did in Exercise 1.

-sandbox
## Conclusion - Exercise 4

We will use a **decision tree model** as our proposed solution.  Our label will be **'lifestyle_recode'**.

-sandbox
## Exercise 5

Summary: 
* Explore your data. 
* Specify which tables and columns will be used for your label/output variable and your feature variables.

**Hint:** Consider aggregating features from other tables.

In [0]:
%python

# Let's code the 'country' column as 1="United States", 0="Canada"
ht_users_pandas_df['country_recode'] = ht_users_pandas_df['country'].apply(lambda x: 1 if x == 'United States' else 0)
ht_users_pandas_df.drop(['country'],axis=1,inplace=True)
ht_users_pandas_df.head()

Out[52]:

,device_id,lifestyle_recode,country_recode
0,01c99836-e48b-11ea-8204-0242ac110002,1,1
1,d27af598-e48a-11ea-8204-0242ac110002,1,1
2,d2f7cdb6-e48a-11ea-8204-0242ac110002,1,1
3,d38f5cf8-e48a-11ea-8204-0242ac110002,1,1
4,d465e746-e48a-11ea-8204-0242ac110002,1,1


In [0]:
ht_agg_pandas_df.head()

Out[53]:

,device_id,mean_bmi,mean_active_heartrate,mean_resting_heartrate,mean_vo2,mean_steps,lifestyle
0,e9ca5040-e48a-11ea-8204-0242ac110002,12.654054,108.286159,54.028433,33.039727,15997.964384,Cardio Enthusiast
1,da17592c-e48a-11ea-8204-0242ac110002,20.913669,108.150718,54.779083,36.500396,12811.049315,Cardio Enthusiast
2,f88d1252-e48a-11ea-8204-0242ac110002,21.775090,123.756602,82.717093,18.609143,10444.443836,Athlete
3,edc57fb2-e48a-11ea-8204-0242ac110002,10.975399,110.260624,54.656832,29.183789,16649.802740,Cardio Enthusiast
4,f8fd7682-e48a-11ea-8204-0242ac110002,22.338725,131.062285,81.114548,24.338431,5219.717808,Sedentary


**Let's join 'ht_users_pandas_df' with 'ht_agg_pandas_df' on 'device_id' and drop the 'lifestyle' and 'device_id' columns...**

In [0]:
merged_df = ht_users_pandas_df.merge(ht_agg_pandas_df, on='device_id').drop(['lifestyle','device_id'], axis=1)
merged_df.head()

Out[55]:

,lifestyle_recode,country_recode,mean_bmi,mean_active_heartrate,mean_resting_heartrate,mean_vo2,mean_steps
0,1,1,24.278096,129.179050,76.781773,23.544670,10212.205479
1,1,1,24.765114,113.886581,59.039049,33.984157,12497.750685
2,1,1,26.324105,112.977027,60.144931,33.163264,12026.394521
3,1,1,28.221315,130.748391,60.391901,31.906229,7181.635616
4,1,1,24.321049,128.973500,60.331768,33.699063,10830.093151


-sandbox
## Conclusion - Exercise 5

Our dataset for modeling will include features from both 'ht_users_pandas_df' and 'ht_agg_pandas_df' datasets through a merge (join) on the common feature of 'device_id'.  The 'X' matrix will contain the features ['country_recode', 'mean_bmi', 'mean_active_heartrate', 'mean_resting_heartrate', 'mean_vo2', 'mean_steps']. The 'y' or label vector will be ['lifestyle_recode'].

-sandbox
## Exercise 6

Summary: 
* Prepare your modeling data. 
* Create a customer-level modeling table with the correct output variable and features. 
* Finally, split your data between training and test sets. Make sure this split aligns with that of your baseline solution.

**Hint:** Consider how to make the data split reproducible.

In [0]:
%python
y = merged_df['lifestyle_recode']
X = merged_df.drop(['lifestyle_recode'],axis=1)  

# Create the train and test datasets and ensure that the data split is reproducible by including a 'random_state' attribute.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

-sandbox
## Exercise 7

Summary: 
* Build the model specified in your answer to Exercise 4. 
* Be sure to use an evaluation metric that aligns with your specified objective.

**Hint:** This evaluation metric should align with the one used in your baseline solution.

In [0]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()

# Train the decision tree model on X_train and y_train
dt_model.fit(X_train,y_train)


Out[57]: DecisionTreeClassifier()

-sandbox
## Exercise 8

Summary: 
* Assess your model against the overall objective. 
* Be sure to use an evaluation metric that aligns with your specified objective.

**Hint:** Remember that we assess our models against our test data set to ensure that our solutions generalize.

**Hint:** If your solution doesn't meet the objective, consider tweaking the model and data used by the model until it does meet the objective.

In [0]:
from sklearn.metrics import accuracy_score

y_train_predicted = dt_model.predict(X_train)
print("training accuracy: ", accuracy_score(y_train, y_train_predicted))

y_test_predicted = dt_model.predict(X_test)
print("test accuracy: ", accuracy_score(y_test, y_test_predicted))


training accuracy: 1.0
test accuracy: 1.0

After completing all of the above objectives, you should be ready to communicate your results. Move to the next video in the lesson for a description on that part of the project.

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>